In [21]:
# Import all libraries used for Tneserflow

import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import keras
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import optimizers
import pathlib
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Flatten, Conv2D, MaxPool2D, Activation, Dense, Dropout
from keras.optimizers import Adam
from keras import regularizers, optimizers
from keras.preprocessing.image import ImageDataGenerator

In [13]:
# Import data from folders and load into the system
train_path = os.getcwd() +'/data/train_set/train_set'
test_path = os.getcwd() +'/data/test_set'
train_csv_path = os.getcwd() + '/train_labels.csv'
test_csv_path = os.getcwd() + '/sample.csv'

train_csv = pd.read_csv(train_csv_path)
train_csv.head()

test_csv = pd.read_csv(test_csv_path)
test_csv.head()

img_height, img_width = 256, 256

train_csv['label'] = train_csv['label'].astype(str)
test_csv['label'] = test_csv['label'].astype(str)
    
train_datagen = ImageDataGenerator(
    rescale=1./255,
#     validation_split=0.15,
#     shear_range=0.2,
#     zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_dataframe(
    train_csv,
    directory = train_path,
    subset = None,
    x_col = 'img_name',
    y_col = 'label',
    color_mode = 'grayscale',
    target_size = (256, 256),
    class_mode = 'categorical'
)

val_generator = train_datagen.flow_from_dataframe(
    test_csv,
    directory = test_path,
    subset = None,
    x_col = 'img_name',
    y_col = 'label',
    color_mode = 'grayscale',
    target_size = (256, 256),
    class_mode = 'categorical'
)


Found 30612 validated image filenames belonging to 80 classes.
Found 7653 validated image filenames belonging to 1 classes.


In [5]:
# TODO Display 9 train images with their labels

plt.figure(figsize=(10, 10))
# plt.imshow(train_generator[0][0])

# for images in train_generator:
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i])
#         plt.title(train_csv.label[i])
#         plt.axis("off")

print(train_generator[0][0].shape)

for image_batch, labels_batch in train_generator:
    print(image_batch.shape)
    print(labels_batch.shape)
    break


(32, 256, 256, 1)
(32, 256, 256, 1)
(32, 80)


<Figure size 720x720 with 0 Axes>

In [7]:
# Normalize the data
# normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

# normalized_ds = train_generator.map(lambda x: (normalization_layer(x)))

# print(normalized_ds.take(1))

# for i in normalized_ds.take(1):
#     print('e')

In [39]:
num_classes = 3

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 1)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(80, activation='relu'),
#   layers.Dense(num_classes)
])

In [40]:
#Compile the model
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])


model.compile(optimizer=Adam(lr=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_9 (Rescaling)      (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 256, 256, 16)      160       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 32, 32, 64)       

In [42]:
# Create an dataset

print(train_generator)

In [ ]:
epochs=10
# MOET DATASET MAKEN MET LABLES 

history = model.fit(
    train_generator,
    steps_per_epoch = 2000,
    epochs = 10,
    validation_data = val_generator,
    validation_steps = 64
)

Epoch 1/10
 624/2000 [========>.....................] - ETA: 34:15 - loss: 14.2568 - accuracy: 0.0173